In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [9]:
import weasyprint as wp
import jinja2
import calendar

from datahandling import *
from graphing import *

import IPython.core.display as ipd

mpl.style.use('seaborn-bright')#'fivethirtyeight')
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['figure.figsize'] = (8,12) #(3,2)
mpl.rcParams['axes.titlesize'] = 'large'
mpl.rcParams['axes.labelsize'] = 'small'
mpl.rcParams['xtick.labelsize'] = 'small'
mpl.rcParams['ytick.labelsize'] = 'small'
mpl.rcParams['legend.fontsize'] = 'small'

In [3]:
pd.set_option('chained_assignment', None) # Hush up, SettingWithCopyWarning
df = readfile("./testdata/fetch.bax")
t_start, t_end = (df.index.min(), df.index.max())
dfs = split_by_id(df)
names = unique_sensors(df)

dfs = fix_humidity(dfs)
dfs = diff_pir(dfs)
dfs = fix_temp(dfs)

In [10]:
%%capture
types=[("Temp", "Temperature ˚C"), ("Humidity", "Humidity %RH"), ("Light", "Light (lux)")]
figs=[ weekly_graph( dfs, *t, t_start, t_end ) for t in types ]
series,labels=zip(*types)

to_plot = [{'type':t,'label':l,'data':d} for t,l,d in zip(series, labels, figs)]

In [11]:
import base64
print('\n'.join(base64.b64decode(to_plot[0]['data']).decode('utf8').split('\n')[4:7])[:100])
#ipd.HTML("<img src='data:image/svg+xml;charset=utf-8;base64,%s' />" % figs[0])
ipd.HTML(base64.b64decode(to_plot[0]['data']).decode('utf8'))

<svg height="634pt" version="1.1" viewBox="0 0 475 634" width="475pt" xmlns="http://www.w3.org/2000/


In [12]:
report_template = '''
{% for p in to_plot %}
<h2>{{p.label}} Report: {{location}}</h1>
<h3>Time period: <em>{{t_start}} to {{t_end}}</em></h3>

<figure>
    <img src="data:image/svg+xml;charset=utf-8;base64,{{p.data}}" />
    <!--<figcaption></figcaption>-->
</figure>
{% endfor %}
'''

print_css = wp.CSS(string='''
@page { size: A4 portrait; margin:1cm 2cm; }
html { font-family: sans; }
html, body {
  width: 100%;
  margin: 0;
  padding: 0;
  position: relative;
}
figure {
    padding:0;
    margin:0;
    width:100%;
}
img { width: 100%; }
''')
#page-break-before # for later

debug_css = wp.CSS(string='''
* { box-sizing: border-box; outline: solid 2px #900; }
* > * { outline-color: #090; }
* > * > * { outline-color: #009; }
''')

variables = {
    'location' : 'Open Lab',
    't_start': t_start, 
    't_end'  : t_end,
    'to_plot' : to_plot
}
output = jinja2.Template(report_template).render(**variables)

In [13]:
# with open('test.htm', 'w+') as t: t.write(output)
#ipd.display(ipd.HTML(output))

# write to PDF
htm = wp.HTML(string=output, base_url='.')
pdf = htm.write_pdf(target='weasyprint_output.pdf', zoom=2, stylesheets=[print_css])#, debug_css])

!open weasyprint_output.pdf